# Exemplo Avançado 1 – Coleta Automática de Preços de Ações com Python

Este notebook coleta automaticamente os preços de ações da B3 utilizando o Yahoo Finance e armazena os dados em um arquivo CSV. O processo é contínuo e ocorre a cada 15 minutos, permitindo a construção de um histórico de preços ao longo do dia.

---

## Objetivo:

- Buscar os preços das ações (ex: PETR4.SA, VALE3.SA)
- Armazenar os dados com horário da coleta
- Repetir esse processo a cada 15 minutos automaticamente

## Etapa 1 – Bibliotecas utilizadas

| Biblioteca      | Função                                                                 |
|----------------|------------------------------------------------------------------------|
| `yfinance`      | Buscar preços das ações diretamente do Yahoo Finance                   |
| `pandas`        | Organizar os dados em tabelas e salvar em CSV                          |
| `schedule`      | Agendar tarefas automáticas de coleta a cada X minutos                 |
| `datetime`      | Gerar data e hora da coleta                                            |
| `time`          | Controlar o tempo de espera entre coletas                              |
| `os`            | Verificar se o arquivo já existe para atualizar sem sobrescrever       |

In [ ]:
!pip install schedule --quiet

In [ ]:
# Instale o yfinance caso ainda não tenha
!pip install yfinance --quiet

import yfinance as yf
import pandas as pd
import time
import schedule
import os
from datetime import datetime


## Etapa 2 – Configuração inicial

Definimos:
- Quais ações vamos monitorar
- O nome do arquivo onde os dados serão armazenados


In [ ]:
acoes = ['PETR4.SA', 'VALE3.SA', 'ITUB4.SA']  # Ações que queremos monitorar
arquivo_saida = 'dados_acoes.csv'            # Nome do arquivo que guardará os dados

## Etapa 3 – Função de coleta de dados

Essa função:
- Coleta o preço mais recente de cada ação
- Registra o horário da coleta
- Salva ou atualiza o arquivo `dados_acoes.csv`


In [15]:
def coletar_dados():
    agora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    dados = []

    for acao in acoes:
        info = yf.Ticker(acao).history(period='1d', interval='1m')
        if not info.empty:
            preco_atual = info['Close'].iloc[-1]
            dados.append({'timestamp': agora, 'acao': acao, 'preco': preco_atual})

    df_novo = pd.DataFrame(dados)

    if os.path.exists(arquivo_saida):
        df_existente = pd.read_csv(arquivo_saida)
        df_total = pd.concat([df_existente, df_novo], ignore_index=True)
    else:
        df_total = df_novo

    df_total.to_csv(arquivo_saida, index=False)
    print(f"[{agora}] Dados coletados e salvos.")


## Etapa 4 – Agendamento da coleta

Vamos configurar para coletar dados a cada 15 minutos automaticamente.

In [16]:
schedule.every(15).minutes.do(coletar_dados)

print("⏳ Iniciando o monitoramento. Pressione STOP para encerrar a execução.")

while True:
    schedule.run_pending()
    time.sleep(60)

⏳ Iniciando o monitoramento. Pressione STOP para encerrar a execução.
[2025-06-08 15:19:06] Dados coletados e salvos.
[2025-06-08 15:20:07] Dados coletados e salvos.
[2025-06-08 15:21:08] Dados coletados e salvos.


KeyboardInterrupt: 

In [17]:
pd.read_csv('dados_acoes.csv')

,timestamp,acao,preco
0,2025-06-08 15:11:16,PETR4.SA,29.690001
1,2025-06-08 15:11:16,VALE3.SA,52.930000
2,2025-06-08 15:11:16,ITUB4.SA,36.520000
3,2025-06-08 15:12:39,PETR4.SA,29.690001
4,2025-06-08 15:12:39,VALE3.SA,52.930000
5,2025-06-08 15:12:39,ITUB4.SA,36.520000
6,2025-06-08 15:19:06,PETR4.SA,29.690001
7,2025-06-08 15:19:06,VALE3.SA,52.930000
8,2025-06-08 15:19:06,ITUB4.SA,36.520000
9,2025-06-08 15:20:07,PETR4.SA,29.690001
